In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager
import numpy as np
import pandas as pd
from time import sleep
from random import randint
import datetime

In [2]:
# Set up Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
city = 'Boston'

# Create empty list to store scraped data
datatoget = []

for i in range(0,8):    
    
    # Set url
    url = f'https://boston.craigslist.org/search/apa#search=1~thumb~{i}~0'
    browser.visit(url)
    sleep(randint(5,7))
    
    # Parse the HTML
    html = browser.html
    soup = bs(html, 'html.parser')

    # find and extract relevant data
    search = soup.find_all('div', class_='result-info')

    print('\n================')
    print(f'This is page {i+1}')
    print('================')    
    
    # Loop through data found and extract relevant information
    for (counter,link) in enumerate(search):

        # Scrape dates with error handeling
        try:
            dates = link.find_all('div', class_='meta')[0].text.split('·')[0]

            current_date = datetime.datetime.now()

            if link.find_all('div', class_='meta')[0].text.split('·')[0][-9:] == 'hours ago':
                dates = current_date.strftime(f'{current_date.month}/{current_date.day}')

            elif link.find_all('div', class_='meta')[0].text.split('·')[0][-11:] == 'minutes ago':
                dates = current_date.strftime(f'{current_date.month}/{current_date.day}')
            
            elif link.find_all('div', class_='meta')[0].text.split('·')[0][-8:] == 'hour ago':
                dates = current_date.strftime(f'{current_date.month}/{current_date.day}')

        except:
            dates = np.nan

        # Scrape titles
        titles = link.find_all('a', class_='titlestring')[0].text

        # Scrape links
        links = link.find_all('a', class_='titlestring', href=True)[0]['href']

        # Scrape prices with error handeling
        try:
            prices = link.find_all('span', class_='priceinfo')[0].text
            
        except:
            prices = np.nan

        # Scrape neighborhoods with error handeling
        try:
            neighborhoods = link.find('div', class_='supertitle').text
        
        except:
            neighborhoods = np.nan

        # Visit each link seperately
        browser.visit(links)
        
        # Ensures that the server does not get overloaded with requests
        sleep(randint(2,5)) 

        # After visiting link, extract bath and amenities data
        html = browser.html
        soup = bs(html, 'html.parser')
  
        # Error handeling in case there are no sqft displayed
        try:
            if soup.find_all('p', class_='attrgroup')[0].text.split('\n')[2][-3:] == 'ft2':
                sqft = soup.find_all('p', class_='attrgroup')[0].text.split('\n')[2][:-3]
            else:
                sqft = np.nan
        except:
            sqft = np.nan
        
        # Error handeling in case there are no bedrooms and bathrooms displayed
        try:
            bedrooms = soup.find_all('p', class_='attrgroup')[0].text.split('\n')[1].split('/')[0].strip().lower()

        except:
            bedrooms = np.nan
        
        try:
            baths = soup.find_all('p', class_='attrgroup')[0].text.split('\n')[1].split()[2][:-2]

        except:
            baths = np.nan


        amenities = soup.find_all('p', class_='attrgroup')[1].text.split('\n')
        # List comprehension used to make sure no empty strings get added to the list
        all_amenities = [am for am in amenities if len(am) >= 1]

        # Print statements to ensure that the correct data is scraped
        print(f'\nListing Number: {counter+1}')
        print(f'Square Footage: {sqft if sqft == sqft else "not available"}')
        print(f'Price: {prices if prices == prices else "not available"}')
        print(f'Title: {titles}')
        print(f'Date: {dates}')
        print(f'Number of Bedrooms: {bedrooms if bedrooms == bedrooms else "not available"}')
        print(f'Neighborhood: {neighborhoods if neighborhoods == neighborhoods else "not available"}')
        print(f'Number of Bathrooms: {baths if baths == baths else "not available"}')
        print(f'Link: {links}')
        print(f'Amenities: {all_amenities}')

        # Create dicitonary for our DataFrame
        data = {'date': dates,
                'title': titles,
                'link': links,
                'price': prices,
                'bedroom': bedrooms,
                'sqft': sqft,
                'neighborhood': neighborhoods,
                'bathroom': baths,
                'amenities': all_amenities}

        # Append the data
        datatoget.append(data)

print('\n================ Done ================')

# Quit automated browser instance
browser.quit()    


This is page 1

Listing Number: 1
Square Footage: 1200
Price: $1
Title: Deleaded 3 Bedroom Apartment
Date: 1/17
Number of Bedrooms: 3br
Neighborhood: Dorchester
Number of Bathrooms: 1
Link: https://boston.craigslist.org/gbs/apa/d/dorchester-center-deleaded-bedroom/7579366193.html
Amenities: ['apartment', 'no laundry on site', 'off-street parking', 'rent period: monthly']

Listing Number: 2
Square Footage: not available
Price: $3,025
Title: AMAZING Cambridge 1 Bed- Brand New!
Date: 1/17
Number of Bedrooms: 1br
Neighborhood: Cambridge
Number of Bathrooms: 1
Link: https://boston.craigslist.org/gbs/apa/d/cambridge-amazing-cambridge-bed-brand/7579362991.html
Amenities: ['cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'laundry in bldg', 'off-street parking', 'rent period: monthly']

Listing Number: 3
Square Footage: not available
Price: $2,350
Title: ➤ Brighton 2 Bedroom ➤ Close to the Green Line ➤ NO BROKER FEE
Date: 1/17
Number of Bedrooms: 2br
Neighborhood: Brighton
Number of 


Listing Number: 20
Square Footage: not available
Price: $2,595
Title: 2 BED 2 BATH || HEAT/HW/PRKG || LARGE CLOSETS || NO LAST-NO SECURITY!
Date: 1/16
Number of Bedrooms: 2br
Neighborhood: NEWTON
Number of Bathrooms: 2
Link: https://boston.craigslist.org/gbs/apa/d/newtonville-bed-bath-heat-hw-prkg-large/7579308455.html
Amenities: ['broker fee details: 2595', 'apartment', 'laundry in bldg', 'listed by: FOLEY & SOULE ASSOCIATES', 'off-street parking', 'rent period: monthly']

Listing Number: 21
Square Footage: not available
Price: $2,495
Title: LOWER FALLS 2 BED *** HEAT/HW/PRKG *** WALK TO RIVERSIDE T ***
Date: 1/16
Number of Bedrooms: 2br
Neighborhood: NEWTON
Number of Bathrooms: 1
Link: https://boston.craigslist.org/gbs/apa/d/newton-lower-falls-lower-falls-bed-heat/7579308262.html
Amenities: ['broker fee details: 2495', 'apartment', 'laundry in bldg', 'listed by: FOLEY & SOULE ASSOCIATES', 'off-street parking', 'rent period: monthly']

Listing Number: 22
Square Footage: 730
Price: $3


Listing Number: 39
Square Footage: not available
Price: $2,495
Title: 2 Bd on Ashford St., Parking For Rent
Date: 1/16
Number of Bedrooms: 2br
Neighborhood: Allston
Number of Bathrooms: 1
Link: https://boston.craigslist.org/gbs/apa/d/allston-bd-on-ashford-st-parking-for/7579277791.html
Amenities: ['cats are OK - purrr', 'condo', 'laundry in bldg', 'no smoking', 'off-street parking', 'rent period: monthly']

Listing Number: 40
Square Footage: 640
Price: $2,623
Title: ✨STATE OF THE ART WATERFRONT LUXURY 1BED, W/D, GYM, ROOFDECK! ✨
Date: 1/16
Number of Bedrooms: 1br
Neighborhood: Boston Waterfront by Walk to Subway
Number of Bathrooms: 1
Link: https://boston.craigslist.org/gbs/apa/d/east-boston-state-of-the-art-waterfront/7579275096.html
Amenities: ['EV charging', 'air conditioning', 'cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d in unit', 'no smoking', 'attached garage', 'rent period: monthly']

Listing Number: 41
Square Footage: 475
Price: $2,445
Title: ✨IMPRESSIVE NEW


Listing Number: 58
Square Footage: 496
Price: $2,200
Title: 🔥 Top floor | Heat included | elevator | 😸 friendly
Date: 1/16
Number of Bedrooms: 0br
Neighborhood: Cambridge
Number of Bathrooms: 1
Link: https://boston.craigslist.org/gbs/apa/d/cambridge-top-floor-heat-included/7579207740.html
Amenities: ["broker fee details: Broker's Fee equals 1 Month's Rent", 'cats are OK - purrr', 'apartment', 'laundry on site', 'listed by: Apartment Rental Experts', 'no smoking', 'street parking', 'rent period: monthly']

Listing Number: 59
Square Footage: not available
Price: $2,600
Title: Cambridge/Central Square 2 Bedroom
Date: 1/16
Number of Bedrooms: 2br
Neighborhood: Cambridge \ Central Sq. \ MIT
Number of Bathrooms: 1.5
Link: https://boston.craigslist.org/gbs/apa/d/cambridge-cambridge-central-square/7579206488.html
Amenities: ['cats are OK - purrr', 'apartment', 'laundry on site', 'no smoking', 'off-street parking', 'rent period: monthly']

Listing Number: 60
Square Footage: 855
Price: $2,200
T


Listing Number: 77
Square Footage: 920
Price: $2,395
Title: Spacious two bedroom/TWO full bathrooms
Date: 1/16
Number of Bedrooms: 2br
Neighborhood: Quincy
Number of Bathrooms: 2
Link: https://boston.craigslist.org/sob/apa/d/quincy-spacious-two-bedroom-two-full/7579138286.html
Amenities: ['air conditioning', 'apartment', 'laundry in bldg', 'no smoking', 'attached garage', 'rent period: monthly']

Listing Number: 78
Square Footage: not available
Price: $2,800
Title: Spring Hill/Davis Sq 2BR - Complete Reno - Hwd Floors - Dishwasher
Date: 1/16
Number of Bedrooms: 2br
Neighborhood: Somerville \ Spring Hill \ Davis Sq.
Number of Bathrooms: 1
Link: https://boston.craigslist.org/gbs/apa/d/somerville-spring-hill-davis-sq-2br/7579138201.html
Amenities: ['cats are OK - purrr', 'apartment', 'laundry on site', 'no smoking', 'off-street parking', 'rent period: monthly']

Listing Number: 79
Square Footage: 800
Price: $2,095
Title: First month FREE if moved in by Feb 1
Date: 1/16
Number of Bedrooms

...


Listing Number: 36
Square Footage: 1010
Price: $4,498
Title: 2BR UNIT CLOSE TO SULLIVAN STATION! WITH W&D IN UNIT! GREAT DEAL!
Date: 1/9
Number of Bedrooms: 2br
Neighborhood: Charlestown
Number of Bathrooms: 2
Link: https://boston.craigslist.org/gbs/apa/d/charlestown-2br-unit-close-to-sullivan/7576483766.html
Amenities: ['air conditioning', 'broker fee details: 0', 'cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d in unit', 'listed by: Hartley Realty Group', 'no smoking', 'attached garage', 'rent period: monthly', 'wheelchair accessible']

Listing Number: 37
Square Footage: 860
Price: $3,971
Title: 2BR UNIT CLOSE TO SULLIVAN STATION! WITH W&D IN UNIT! GREAT DEAL!
Date: 1/9
Number of Bedrooms: 2br
Neighborhood: Charlestown
Number of Bathrooms: 2
Link: https://boston.craigslist.org/gbs/apa/d/charlestown-2br-unit-close-to-sullivan/7576483654.html
Amenities: ['air conditioning', 'broker fee details: 0', 'cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d in unit'


Listing Number: 54
Square Footage: 4500
Price: $1,000
Title: Bedrooms for rent in gorgeous single-family home NO FEE
Date: 1/8
Number of Bedrooms: 5br
Neighborhood: Hyde Park, Boston
Number of Bathrooms: 2
Link: https://boston.craigslist.org/gbs/apa/d/hyde-park-bedrooms-for-rent-in-gorgeous/7576361786.html
Amenities: ['house', 'w/d in unit', 'off-street parking', 'rent period: monthly']

Listing Number: 55
Square Footage: not available
Price: $3,000
Title: Really Nice 1st Floor Apt, Washer & Dyer in Unit, Hardwood Fls
Date: 1/8
Number of Bedrooms: 3br
Neighborhood: Dorchester
Number of Bathrooms: 1
Link: https://boston.craigslist.org/gbs/apa/d/dorchester-center-really-nice-1st-floor/7576330782.html
Amenities: ['apartment', 'w/d in unit', 'street parking', 'rent period: monthly']

Listing Number: 56
Square Footage: 625
Price: $1,150
Title: 2-bedroom apt in Clinton
Date: 1/8
Number of Bedrooms: 2br
Neighborhood: Clinton (Metrowest)
Number of Bathrooms: 1
Link: https://boston.craigslist.


Listing Number: 73
Square Footage: 2200
Price: $5,800
Title: For Rent, S.F., 9 rooms, 5 bedrooms, 2 baths, 2 car garage
Date: 1/8
Number of Bedrooms: 5br
Neighborhood: Lynnfield
Number of Bathrooms: 2
Link: https://boston.craigslist.org/nos/apa/d/lynnfield-for-rent-sf-rooms-bedrooms/7576222134.html
Amenities: ['application fee details: 100', 'broker fee details: 2900', 'house', 'w/d hookups', 'listed by: Bill Kafkas RE', 'attached garage', 'rent period: monthly']

Listing Number: 74
Square Footage: 950
Price: $2,350
Title: 2 bedroom / 1 bathroom - sunny updated apartment in 2 family
Date: 1/8
Number of Bedrooms: 2br
Neighborhood: Melrose
Number of Bathrooms: 1
Link: https://boston.craigslist.org/nos/apa/d/melrose-bedroom-bathroom-sunny-updated/7576220490.html
Amenities: ['apartment', 'laundry in bldg', 'off-street parking', 'rent period: monthly']

Listing Number: 75
Square Footage: 725
Price: $3,200
Title: Sunsplashed 1BR condo in South End (patio, in unit W/D, dishwasher)
Date: 1/8



Listing Number: 93
Square Footage: not available
Price: $1,900
Title: 1 BEDROOM..NATICK..COMMUTER RAIL..POOL,ETC
Date: 1/7
Number of Bedrooms: 1br
Neighborhood: NATICK
Number of Bathrooms: 1
Link: https://boston.craigslist.org/bmw/apa/d/natick-bedroomnatickcommuter-railpooletc/7576058122.html
Amenities: ['air conditioning', 'broker fee details: 1900', 'apartment', 'w/d in unit', 'listed by: BONNIE', 'no smoking', 'off-street parking', 'rent period: monthly']

Listing Number: 94
Square Footage: 2200
Price: $5,000
Title: HARVARD SINGLE FAMILY 3 BEDS 3 BATHS AND A LOFT...PETS OK
Date: 1/7
Number of Bedrooms: 3br
Neighborhood: CAMBRIDGE/HURON VILLAGE
Number of Bathrooms: 3
Link: https://boston.craigslist.org/gbs/apa/d/cambridge-harvard-single-family-beds/7576057932.html
Amenities: ['broker fee details: 5000', 'cats are OK - purrr', 'dogs are OK - wooof', 'house', 'w/d in unit', 'listed by: BONNIE', 'no smoking', 'off-street parking', 'rent period: monthly']

Listing Number: 95
Square Foot


Listing Number: 112
Square Footage: not available
Price: $2,632
Title: 1.5 FREE MONTHS* & No Fee! Brand New Apts With Laundry & Central Air!
Date: 1/7
Number of Bedrooms: 1br
Neighborhood: Jamaica Plain
Number of Bathrooms: 1
Link: https://boston.craigslist.org/gbs/apa/d/jamaica-plain-15-free-months-no-fee/7575920472.html
Amenities: ['apartment', 'w/d in unit', 'off-street parking', 'rent period: monthly']

Listing Number: 113
Square Footage: not available
Price: $2,632
Title: 1.5 FREE MONTHS* & No Fee! Brand New Apts With Laundry & Central Air!
Date: 1/7
Number of Bedrooms: 1br
Neighborhood: Boston
Number of Bathrooms: 1
Link: https://boston.craigslist.org/gbs/apa/d/hyde-park-15-free-months-no-fee-brand/7575919025.html
Amenities: ['apartment', 'w/d in unit', 'off-street parking', 'rent period: monthly']

Listing Number: 114
Square Footage: 750
Price: $3,900
Title: 2 Bedroom Beacon Hill Apartment
Date: 1/7
Number of Bedrooms: 2br
Neighborhood: Beacon Hill
Number of Bathrooms: 1
Link: 

In [4]:
# creating DataFrame
df = pd.DataFrame(datatoget)
df

,date,title,link,price,bedroom,sqft,neighborhood,bathroom,amenities
0,1/17,Deleaded 3 Bedroom Apartment,https://boston.craigslist.org/gbs/apa/d/dorche...,$1,3br,1200,Dorchester,1,"[apartment, no laundry on site, off-street par..."
1,1/17,AMAZING Cambridge 1 Bed- Brand New!,https://boston.craigslist.org/gbs/apa/d/cambri...,"$3,025",1br,NaN,Cambridge,1,"[cats are OK - purrr, dogs are OK - wooof, apa..."
2,1/17,➤ Brighton 2 Bedroom ➤ Close to the Green Line...,https://boston.craigslist.org/gbs/apa/d/bright...,"$2,350",2br,NaN,Brighton,1,"[apartment, laundry in bldg, street parking, r..."
3,1/17,AMAZING MEDFORD 2 BED 2 BATH,https://boston.craigslist.org/gbs/apa/d/somerv...,"$3,000",2br,NaN,MEDFORD,2,"[apartment, w/d in unit, off-street parking, r..."
4,1/17,"Available for rent now, 2bed2bath apartment pr...",https://boston.craigslist.org/gbs/apa/d/stoneh...,"$2,400",2br,1050,"Stoneham, MA",2,"[air conditioning, cats are OK - purrr, dogs a..."
...,...,...,...,...,...,...,...,...,...
955,1/7,Amazing Location - Close to MIT - Move in Jan. 15,https://boston.craigslist.org/gbs/apa/d/cambri...,"$2,000",1br,700,Cambridge,1,"[broker fee details: $2000, apartment, no laun..."
956,1/7,Just what you need! Washer/dryer in this 809 s...,https://boston.craigslist.org/gbs/apa/d/bellin...,"$1,944",1br,809,Bellingham,1,"[cats are OK - purrr, dogs are OK - wooof, apa..."
957,1/7,➤NORTHEASTERN BERKLEE FENWAY 1 BD 1 BATH INC A...,https://boston.craigslist.org/gbs/apa/d/boston...,"$2,865",1br,NaN,Fenway,1,"[cats are OK - purrr, apartment, laundry in bl..."
958,1/7,"Beautiful 2bd, 2ba, 2parking, condo available ...",https://boston.craigslist.org/gbs/apa/d/east-b...,"$3,200",2br,1276,East Boston,2,"[air conditioning, apartment, w/d in unit, no ..."


In [5]:
# Exporting data to csv
df.to_csv(f"{city}_data_raw.csv", index=False)